In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import caiman
# import wrappers.caiman_wrapper as cm
# import wrappers.suite2p_wrapper as s2p

from caiman.source_extraction.cnmf.params import CNMFParams
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.motion_correction import MotionCorrect

info = {}

In [ ]:
file_path = os.path.join(
    '/Users', 'shogoakiyama', 'caiman_data', 
    'example_movies', 'Sue_2x_3000_40_-46.tif')

In [ ]:
opts = CNMFParams()
motion_opts = opts.get_group('motion')

In [ ]:
mc = MotionCorrect(file_path, dview=None, **motion_opts)
mc.motion_correct(save_movie=True)

In [ ]:
info['total_template_rig'] = mc.total_template_rig
info['shifts_rig'] = mc.shifts_rig
info['border_to_0'] = 0 if mc.border_nan is 'copy' else mc.border_to_0
info['mmap_file'] = mc.mmap_file

In [ ]:
fname_new = caiman.save_memmap(info['mmap_file'], base_name='memmap_', order='C',
                           border_to_0=info['border_to_0'], dview=None) # exclude borders
Yr, dims, T = caiman.load_memmap(fname_new)
info['images'] = np.reshape(Yr.T, [T] + list(dims), order='F') 

### CNMF

In [ ]:
cnm = cnmf.CNMF(1, params=opts, dview=None)
cnm = cnm.fit(info['images'])

In [ ]:
### 輪郭プロット（複数枚の画像から）
#%% plot contours of found components
Cn = caiman.local_correlations(info['images'].transpose(1, 2, 0))
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours_nb(img=Cn)

In [ ]:
info['movies'] = cnm.estimates.play_movie(
    info['images'], q_max=99.9, gain_res=2, magnification=2, 
    bpx=info['border_to_0'], include_bck=False, display=False)